# Extract one Object from Light Curves in single band at AuxTel

- author Sylvie Dagoret-Campagne (from Lauren macArthur)
- creation date 2024-05-28
- last update 2024-06-01

w_2024_16
  
``Quantum Graphs``: https://tigress-web.princeton.edu/~lkelvin/pipelines/ 


https://tigress-web.princeton.edu/~lkelvin/pipelines/current/drp_pipe/LSSTComCamSim/nightly-validation-ops-rehearsal-3/

``CalibrateTask``: 
https://github.com/lsst/pipe_tasks/blob/main/python/lsst/pipe/tasks/calibrate.py#L392-L399


``Shemas``:
https://dm.lsst.org/sdm_schemas/browser/



In [ ]:
from lsst.daf.butler import Butler

import astropy.units as u
import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)
from astropy.time import Time

import scipy.stats

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom

In [ ]:
from collections import OrderedDict

In [ ]:
#xx-small
#x-small
#small
#medium
#large
#x-large
#xx-large

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
saveDir = "./"
doSaveFigs = False  # set to True if you actually want to save the figures in the above dir

In [ ]:
# Set some selections for reliable data
minNumMatches = 3
minSnCalibFlux = 50  # 100

In [ ]:
# INSERT YOUR auxTel/LATISS collection and tract/band HERE
butlerRoot = "/repo/embargo"
# collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20231207/w_2023_49/PREOPS-4648"
#collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED

collectionStr = collection.replace("/", "_")
instrument = "LATISS"
skymapName = "latiss_v1"
#band = "i"
#tract = 5615
band = "g"
tract = 3864

#Aperture flux in instrumental
calibFluxStr = "apFlux_35_0_instFlux"
calibFluxErrStr = "apFlux_35_0_instFluxErr"
calibFluxMagStr = "apFlux_35_0_instMag"
calibFluxMagErrStr = "apFlux_35_0_instMagErr"

#Aperture flux in instrumental
calibFluxStr2 = "apFlux_50_0_instFlux"
calibFluxErrStr2 = "apFlux_50_0_instFluxErr"
calibFluxMagStr2 = "apFlux_50_0_instMag"
calibFluxMagErrStr2 = "apFlux_50_0_instMagErr"

#Aperture flux calibrated calibrated in AB
calibFluxCalStr = "apFlux_35_0_calFlux"
calibFluxCalErrStr = "apFlux_35_0_calFluxErr"
calibFluxCalMagStr = "apFlux_35_0_calMag"
calibFluxCalMagErrStr = "apFlux_35_0_calMagErr"

#Aperture flux calibrated calibrated in AB
calibFluxCalStr2 = "apFlux_50_0_calFlux"
calibFluxCalErrStr2 = "apFlux_50_0_calFluxErr"
calibFluxCalMagStr2 = "apFlux_50_0_calMag"
calibFluxCalMagErrStr2 = "apFlux_50_0_calMagErr"



file_output_selectedsources = f"lightcurves-selectedsources_tract{tract}_band{band}.csv"
fullname_file_output_selectedsources = os.path.join(saveDir,file_output_selectedsources)

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
output_objects_file = f"objectTable-t{tract}-b{band}-{collectionStr}.csv"
output_objects_file_truncated = f"objectTable-t{tract}-b{band}"
print(output_objects_file)

In [ ]:
!ls ../Visits

In [ ]:
file_selected_visits = "../Visits/ccdVisittractpatch_LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"

In [ ]:
df_myselectedvisits = pd.read_csv(file_selected_visits,index_col=0)
my_selectedvisits = list(df_myselectedvisits.index)

In [ ]:
df_myselectedvisits

In [ ]:
# Testing on an HSC RC2 run
# butlerRoot = "/repo/main"
# collection = "HSC/runs/RC2/w_2024_06/DM-42797"

# collectionStr = collection.replace("/", "_")
# instrument = "HSC"
# skymapName = "hsc_rings_v1"
# band = "i"
# tract = 9813

# calibFluxStr = "apFlux_12_0_instFlux"

In [ ]:
# Initiate butler from variables set above
butler = Butler(butlerRoot, collections=collection, instrument=instrument, skymap=skymapName)
camera = butler.get("camera", instrument=instrument)
skymap = butler.get("skyMap")
print("camera_name = {}".format(camera.getName()))
print("collection = {}".format(collection))

In [ ]:
# Try to get the Schema
data_product = "isolated_star_sources"
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collection, where= "instrument='LATISS'")
for i, ref in enumerate(datasetRefs):
    print(i,ref)
    butler_data = butler.get(ref)
    break

if not isinstance(butler_data, pd.core.frame.DataFrame):
    print(butler_data.getSchema())

## isolated_star_sources

The main starting point is the table of isolated_star_sources which has been constructed in step 2a from a catalog
Note that isolated star sources are associated to an object (a static starobject)
Here the work is doneband by band.
It is a good idea to work band by band.

In [ ]:
# Load in isolated_star_sources and trim to band of interest and select the tract
isolatedStarSourcesFull = butler.get("isolated_star_sources", tract=tract)
isolatedStarSourcesFull = isolatedStarSourcesFull[isolatedStarSourcesFull["band"] == band]

In [ ]:
isolatedStarSourcesFull.head()

In [ ]:
isolatedStarSourcesFull.columns

## Select the visits in the preselected list : Desactivated now

I add here my filter to select the visits I want to focus on from file_selected_visits = "../data/202402/SelectedVisits_fall2023_tract_3864.csv"
It include all bands

In [ ]:
#def SelectByVisit(row):
#    if row["visit"] in my_selectedvisits:
#        return True
#    else:
#        return False

In [ ]:
#isolatedStarSourcesFull["flag"] = isolatedStarSourcesFull.apply(SelectByVisit,axis=1,raw=False)

In [ ]:
#isolatedStarSourcesFull = isolatedStarSourcesFull[isolatedStarSourcesFull["flag"]]
#isolatedStarSourcesFull.drop("flag",axis=1,inplace=True)

In [ ]:
isolatedStarSourcesFull

In [ ]:
# Just to have a look at what's in the catalog:
isolatedStarSourcesFull[isolatedStarSourcesFull.index == 0]

In [ ]:
print("calibFluxStr = {},,minSnCalibFlux = {}, minNumMatches = {}".format(calibFluxStr,minSnCalibFlux,minNumMatches))

## Select a number of visits with enough good sources in the required band

- extract the list of known objects

In [ ]:
# Trim the isolated_star_sources cat to those PSF S/N > minSnCalibFlux
# and number of matches sources > minNumMatches.
objIndexListFull = list(set(isolatedStarSourcesFull["obj_index"].values))

# Select the bright stars above a minimum of S/N ratio
if minSnCalibFlux is not None:
    snCalibFlux = isolatedStarSourcesFull[calibFluxStr]/isolatedStarSourcesFull[calibFluxStr + "Err"]
    snMask = snCalibFlux > minSnCalibFlux
    isolatedStarSourcesMinSn = isolatedStarSourcesFull[snMask].copy(deep=True)
else:
    isolatedStarSourcesMinSn = isolatedStarSourcesFull.copy(deep=True)

# object index with obj index in above selected bright selected star
objIndexListMinSn = list(set(isolatedStarSourcesMinSn["obj_index"].values))

# requires that the object is associated to several sources (at least minNumMatches sources)
objIndexList = []
for objIndex in objIndexListMinSn:
    objData = isolatedStarSourcesMinSn[isolatedStarSourcesMinSn["obj_index"] == objIndex]
    if len(objData) >= minNumMatches:
        objIndexList.append(objIndex)
numTrimmed = len(objIndexListFull) - len(objIndexList)

mask = []
for objIndex, visit in zip(isolatedStarSourcesMinSn["obj_index"], isolatedStarSourcesMinSn["visit"]):
    if objIndex in objIndexList:
        mask.append(True)
    else:
        mask.append(False)
isolatedStarSources = isolatedStarSourcesMinSn[mask].copy(deep=True)

# Select a number of visits according quality criteria
visitList = list(set(isolatedStarSources["visit"].values))

if minSnCalibFlux is not None:
    print("Trimmed isolated_star_sources catalog to S/N {} > {} (leaving N={} matched souces from original {})".format(
        calibFluxStr, minSnCalibFlux, len(isolatedStarSources), len(isolatedStarSourcesFull)))
print("Trimmed isolated_star_sources catalog to objecst with nMatches >= {} (leaving {} objects from {}).".format(
    minNumMatches, len(objIndexList), len(objIndexListFull)))

In [ ]:
sorted_visitList = sorted(visitList)
sorted_visitList[:20] 

In [ ]:
def convertVisitToDatestr(visit):

    num = visit//100_000
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return datestr

In [ ]:
def convertVisitToMJD(visit):
    return Time(convertVisitToDatestr(visit)).mjd

In [ ]:
sorted_visits_mjd = list(map(convertVisitToMJD, sorted_visitList))

## sourceTable_visit

This is the main output of FGCM calibration which has selected a number of sources. These sources were originally selected from isolated sources.

In [ ]:
# Extra columns to load from sourceTable_visit catalogs
names = ["psfFlux", "psfFluxErr", "psfFlux_flag", "psfFlux_apCorr", "psfFlux_apCorrErr",
         "extendedness", "detect_isPrimary", "deblend_skipped",
         "gaussianFlux", "gaussianFluxErr", "gaussianFlux_flag",
         "localPhotoCalib", "localPhotoCalibErr", "localPhotoCalib_flag"]

In [ ]:
# Load in the sourceTable_visit catalogs to get the psfFlux and other useful info per source.
# Columns loaded are those in names above.
sourceCatalogs = []
nSourceDict = {}
visitsToRemove = []

# loop over all selected visit
for visit in visitList:
    try:
        sourceCatalogOrig = butler.get("sourceTable_visit", visit=visit, parameters={"columns": names})
        primaryCat = sourceCatalogOrig[sourceCatalogOrig["detect_isPrimary"]].copy(deep=True)
        nSourceDict[visit] = sum(primaryCat["psfFlux"]/primaryCat["psfFluxErr"] > 5)
        sourceCatalogs.append(primaryCat)
    except LookupError:
        print("sourceTable_visit not found for visit {}".format(visit))
        isolatedStarSources = isolatedStarSources[isolatedStarSources["visit"] != visit].copy(deep=True)
        visitsToRemove.append(visit)
visitList = list(set(visitList) - set(visitsToRemove))

# Do the minNumMatches cut again since visits may have been removed due to lack of sourceTable_visit
print("Number of objects with >=3 matches before sourceTable_visit exsistence check: {}".format(len(objIndexList)))
objIndexListAllNew = list(set(isolatedStarSources["obj_index"].values))
objIndexList = []
for objIndex in objIndexListAllNew:
    objData = isolatedStarSources[isolatedStarSources["obj_index"] == objIndex]
    if len(objData) >= minNumMatches:
        objIndexList.append(objIndex)
mask = []
for objIndex, visit in zip(isolatedStarSources["obj_index"], isolatedStarSources["visit"]):
    if objIndex in objIndexList:
        mask.append(True)
    else:
        mask.append(False)
isolatedStarSources = isolatedStarSources[mask].copy(deep=True)
visitList = list(set(isolatedStarSources["visit"].values))
print("Number of objects with >=3 matches after sourceTable_visit exsistence check: {}".format(len(objIndexList)))
dataJoined = pd.concat(sourceCatalogs).merge(isolatedStarSources, on="sourceId", how="inner")

In [ ]:
visitsToRemove 

## Will start to work on dataJoined which associate selected "isolated_star_sources" in the selected band and visits the good calibration in "sourceTable_visit"

### add my info from CCD Visit Table

- CCD visit table is a pre-FGCM calibration
- it provides airmass and initial zero-point

In [ ]:
# need also. ["zeroPoint","airmass"]

In [ ]:
# Collect useful columns from ccdVisitTable
ccdVisitTable = butler.get("ccdVisitTable")
ccdVisitTable["airmass"] = ccdVisitTable["zenithDistance"].apply(lambda x: 1/np.cos(np.pi/180.*x))
ccdVisitTable["medianE"] = np.sqrt(ccdVisitTable["psfStarDeltaE1Median"] ** 2.0 + 
                                    ccdVisitTable["psfStarDeltaE2Median"] ** 2.0)
psfSigmaDict = {}
skyBgDict = {}
skyNoiseDict = {}
expTimeDict = {}
expMidptMjdDict = {}
medianEDict = {}
psfStarScaledDeltaSizeScatterDict = {}
astromOffsetStdDict = {}
psfTraceRadiusDeltaDict = {}
zeroPointDict = {}
airmassDict = {}
seeingDict = {}
npsfStarsDict = {}
psfSigmaDict = {}
# for visit in ccdVisitTable["visitId"].values:
#     if visit in visitList:
for visit in visitList:
    if visit in ccdVisitTable["visitId"].values:
        psfSigmaDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["psfSigma"].values[0]
        skyBgDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["skyBg"].values[0]
        skyNoiseDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["skyNoise"].values[0]
        expTimeDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["expTime"].values[0]
        expMidptMjdDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["expMidptMJD"].values[0]
        medianEDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["medianE"].values[0]
        psfStarScaledDeltaSizeScatterDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["psfStarScaledDeltaSizeScatter"].values[0]
        astromOffsetStdDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["astromOffsetStd"].values[0]
        psfTraceRadiusDeltaDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["psfTraceRadiusDelta"].values[0]
        zeroPointDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["zeroPoint"].values[0]
        airmassDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["airmass"].values[0]
        seeingDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["seeing"].values[0]
        npsfStarsDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["nPsfStar"].values[0]
        psfSigmaDict[visit] = ccdVisitTable[ccdVisitTable["visitId"] == visit]["psfSigma"].values[0]
    else:
        raise RuntimeError("ERROR: visit {} not found in ccdVisitTable".format(visit))

In [ ]:
# Just to have a look at what's in the catalog:
ccdVisitTable[ccdVisitTable.index == ccdVisitTable.index[0]] 

In [ ]:
dataJoined.columns

In [ ]:
calibFluxCalStr

In [ ]:
# Add columns into dataJoined table to have them all in one place
dataJoined["psfSn"] = dataJoined["psfFlux"]/dataJoined["psfFluxErr"]
dataJoined["psfMag"] = (dataJoined["psfFlux"].values*u.nJy).to(u.ABmag).value
dataJoined["psfMagErr"] = 2.5/np.log(10.0)*(dataJoined["psfFluxErr"].values/dataJoined["psfFlux"].values)
# NOTE: psfFlux is the fgcm calibrated flux.  I'm pretty sure you get the "instrumental" 
# flux by dividing psfFlux by the localPhotoCalib value.

dataJoined[calibFluxCalStr] = dataJoined[calibFluxStr]*dataJoined["localPhotoCalib"]
dataJoined[calibFluxCalErrStr] = dataJoined[calibFluxErrStr]*dataJoined["localPhotoCalib"]

dataJoined[calibFluxCalMagStr] = (dataJoined[calibFluxCalStr].values*u.nJy).to(u.ABmag).value
dataJoined[calibFluxCalMagErrStr] = 2.5/np.log(10.0)*(dataJoined[calibFluxCalErrStr].values/dataJoined[calibFluxCalStr].values)

dataJoined[calibFluxCalStr2] = dataJoined[calibFluxStr2]*dataJoined["localPhotoCalib"]
dataJoined[calibFluxCalErrStr2] = dataJoined[calibFluxErrStr2]*dataJoined["localPhotoCalib"]

dataJoined[calibFluxCalMagStr2] = (dataJoined[calibFluxCalStr2].values*u.nJy).to(u.ABmag).value
dataJoined[calibFluxCalMagErrStr2] = 2.5/np.log(10.0)*(dataJoined[calibFluxCalErrStr2].values/dataJoined[calibFluxCalStr2].values)



dataJoined["psfInstMag"] = ((dataJoined["psfFlux"].values/dataJoined["localPhotoCalib"].values)*u.nJy).to(u.ABmag).value
dataJoined["psfInstMagErr"] = 2.5/np.log(10.0)*(dataJoined["psfFluxErr"].values/dataJoined["psfFlux"].values)


dataJoined["psfGausFluxRatio"] = dataJoined["psfFlux"]/dataJoined["gaussianFlux"]

dataJoined["psfSigma"] = dataJoined.apply(lambda x: psfSigmaDict[x["visit"]], axis=1)

dataJoined["skyBg"] = dataJoined.apply(lambda x: skyBgDict[x["visit"]], axis=1)
dataJoined["skyNoise"] = dataJoined.apply(lambda x: skyNoiseDict[x["visit"]], axis=1)

dataJoined["expTime"] = dataJoined.apply(lambda x: expTimeDict[x["visit"]], axis=1)
dataJoined["expMidptMjd"] = dataJoined.apply(lambda x: expMidptMjdDict[x["visit"]], axis=1)

dataJoined["nSource"] = dataJoined.apply(lambda x: nSourceDict[x["visit"]], axis=1)

dataJoined["medianE"] = dataJoined.apply(lambda x: medianEDict[x["visit"]], axis=1)

dataJoined["psfStarScaledDeltaSizeScatter"] = dataJoined.apply(lambda x: psfStarScaledDeltaSizeScatterDict[x["visit"]], axis=1)
dataJoined["astromOffsetStd"] = dataJoined.apply(lambda x: astromOffsetStdDict[x["visit"]], axis=1)
dataJoined["psfTraceRadiusDelta"] = dataJoined.apply(lambda x: psfTraceRadiusDeltaDict[x["visit"]], axis=1)

# I add the airmass and zero-point I want to keep here
dataJoined["zeroPoint"] = dataJoined.apply(lambda x: zeroPointDict[x["visit"]], axis=1)
dataJoined["airmass"] = dataJoined.apply(lambda x: airmassDict[x["visit"]], axis=1)
dataJoined["seeing"] = dataJoined.apply(lambda x: seeingDict[x["visit"]], axis=1)
dataJoined["nPsfStars"] = dataJoined.apply(lambda x: npsfStarsDict[x["visit"]], axis=1)

matchedObjIdList = list(set(dataJoined["obj_index"]))

In [ ]:
dataJoined[dataJoined.index == 0]

In [ ]:
dataJoined.columns

## The association between the independent sources is done through the object index 

- for plotting the repeatability

In [ ]:
dataJoined.columns

In [ ]:
# Compute repeatability related values per object/object group and add
# them inplace to the dataJoined DataFrame.
nSourcesCliAll = []
raSourcesCliAll = []
decSourcesCliAll = []

# in AB magnitudes
psfMagDiffMmagDfList = []
psfMagStdMmagAll = []
psfMagMeanMagAll = []
psfMagMeanMagErrAll = []

psfMagStdMmagDict = OrderedDict()
psfMagMeanMagDict = OrderedDict()  # add the mean magnitude
psfMagDiffChiDfList = []


# aperture in ADU

ap35InstMagMeanMagAll = []
ap35InstMagMeanMagErrAll = []

ap50InstMagMeanMagAll = []
ap50InstMagMeanMagErrAll = []



psfSnAll = []

# in ADU
dataJoined.columns


raDiffMasDfList = []
decDiffMasDfList = []
raCosDecDiffMasDfList = []
objRaDegList = []
objDecDegList = []
raStdMasAll = []
decStdMasAll = []
raCosDecStdMasAll = []
objDataList = OrderedDict()


# loop on objects along ther object index number and compute quantities per object

for objIndex in matchedObjIdList:
    # select sources corresponding to that object
    objData = dataJoined[dataJoined["obj_index"] == objIndex]

    # flux with psf
    psfMag = objData["psfMag"]
    psfMagErr = objData["psfMagErr"]
    psfFluxes = objData['psfFlux']
    psfFluxesErr = objData['psfFluxErr']
    psfSn = psfFluxes/psfFluxesErr
    # average flux over all visits

    clippedMeanMagArray = scipy.stats.sigmaclip(psfMag, low=3.0, high=3.0).clipped
    clippedMeanMag = np.mean(clippedMeanMagArray)
    clippedMeanMagErr = np.std(clippedMeanMagArray)/np.sqrt(len(clippedMeanMagArray))
    nSourcesCli= len(clippedMeanMagArray)

    # flux with aperture
    #apFlux_35_0_instFlux',
    #'apFlux_35_0_instFluxErr',

    ap35InstMag =  -2.5*np.log10(objData["apFlux_35_0_instFlux"])
    #ap35MagErr =  2.5/np.log(10.0)*(objData["apFlux_35_0_instFluxErr"]/objData["apFlux_35_0_instFlux"])
    clippedMeanAp35InstMagArray = scipy.stats.sigmaclip(ap35InstMag, low=3.0, high=3.0).clipped
    clippedMeanAp35InstMag = np.mean(clippedMeanAp35InstMagArray)
    clippedMeanAp35InstMagErr = np.std(clippedMeanAp35InstMagArray)/np.sqrt(len(clippedMeanAp35InstMagArray))
    

    ap50InstMag =  -2.5*np.log10(objData["apFlux_50_0_instFlux"])
    #ap35MagErr =  2.5/np.log(10.0)*(objData["apFlux_35_0_instFluxErr"]/objData["apFlux_35_0_instFlux"])
    clippedMeanAp50InstMagArray = scipy.stats.sigmaclip(ap50InstMag, low=3.0, high=3.0).clipped
    clippedMeanAp50InstMag = np.mean(clippedMeanAp50InstMagArray)
    clippedMeanAp50InstMagErr = np.std(clippedMeanAp50InstMagArray)/np.sqrt(len(clippedMeanAp50InstMagArray))


    # Signal to Noise 
    clippedpsfSnArray = scipy.stats.sigmaclip(psfSn, low=3.0, high=3.0).clipped
    clippedpsfSnMean = np.mean(clippedpsfSnArray)
    
    # array of difference between flux an clipped mean average in mmag
    psfMagDiffMmag = 1000*(psfMag - clippedMeanMag)
    # sigma arrays over the difference
    psfMagStdMmag = np.std(psfMagDiffMmag)
    # residuals array
    psfMagDiffChi = (psfMag - clippedMeanMag)/np.sqrt(psfMagErr**2 + clippedMeanMagErr**2)

    objData["psfMagDiffMmag"] = psfMagDiffMmag
    objData["psfMagDiffChi"] = psfMagDiffChi
    
    clippedMeanRaArray = scipy.stats.sigmaclip(objData["ra"], low=3.0, high=3.0).clipped
    clippedMeanRa = np.mean(clippedMeanRaArray)
    raDiffMas = (objData["ra"] - clippedMeanRa)*3600*1000

    objData["raDiffMas"] = raDiffMas
    
    clippedMeanDecArray = scipy.stats.sigmaclip(objData["dec"], low=3.0, high=3.0).clipped
    clippedMeanDec = np.mean(clippedMeanDecArray)
    decDiffMas = (objData["dec"] - clippedMeanDec)*3600*1000

    objData["decDiffMas"] = decDiffMas
    
    raCosDec = np.deg2rad(objData["ra"])*np.cos(np.deg2rad(objData["dec"]))
    clippedMeanRaCosDecArray = scipy.stats.sigmaclip(raCosDec, low=3.0, high=3.0).clipped
    clippedMeanRaCosDec = np.mean(clippedMeanRaCosDecArray)
    raCosDecDiffRad = (raCosDec - clippedMeanRaCosDec)
    raCosDecDiffMas = np.rad2deg(raCosDecDiffRad)*3600*1000

    objData["raCosDecDiffMas"] = raCosDecDiffMas

    # save for all sources of the object in list
    nSourcesCliAll.append(nSourcesCli)

    raSourcesCliAll.append(clippedMeanRa)
    decSourcesCliAll.append(clippedMeanDec)

    psfMagMeanMagAll.append(clippedMeanMag)
    psfMagMeanMagErrAll.append(clippedMeanMagErr)
    psfSnAll.append(clippedpsfSnMean) 
    
    psfMagDiffMmagDfList.append(psfMagDiffMmag)
    psfMagStdMmagAll.append(psfMagStdMmag)
    psfMagDiffChiDfList.append(psfMagDiffChi)


    #ap35MagDiffMmagDfList = []
    #ap35MagStdMmagAll = []
    ap35InstMagMeanMagAll.append(clippedMeanAp35InstMag)
    ap35InstMagMeanMagErrAll.append(clippedMeanAp35InstMagErr)

    #ap50MagDiffMmagDfList = []
    #ap50MagStdMmagAll = []
    ap50InstMagMeanMagAll.append(clippedMeanAp50InstMag)
    ap50InstMagMeanMagErrAll.append(clippedMeanAp50InstMagErr) 


    raDiffMasDfList.append(raDiffMas)
    decDiffMasDfList.append(decDiffMas)
    raCosDecDiffMasDfList.append(raCosDecDiffMas)
    
    objRaDegList.append(clippedMeanRa)
    objDecDegList.append(clippedMeanDec)

    raStdMas = np.std(raDiffMas)
    raStdMasAll.append(raStdMas)
    decStdMas = np.std(decDiffMas)
    decStdMasAll.append(decStdMas)
    raCosDecStdMas = np.std(raCosDecDiffMas)
    raCosDecStdMasAll.append(raCosDecStdMas)

    # save in Dictionnaries
    psfMagStdMmagDict[objIndex] = psfMagStdMmag
    psfMagMeanMagDict[objIndex] = clippedMeanMag     
    objDataList[objIndex] = objData

# concatenate over all objects - all sources
psfMagDiffMmagDf = pd.concat(psfMagDiffMmagDfList)
psfMagDiffChiDf = pd.concat(psfMagDiffChiDfList)
raDiffMasDf = pd.concat(raDiffMasDfList)
decDiffMasDf = pd.concat(decDiffMasDfList)
raCosDecDiffMasDf = pd.concat(raCosDecDiffMasDfList)

# add this common properties to each source in dataJoined
dataJoined.loc[:, "psfMagDiffMmag"] = psfMagDiffMmagDf
dataJoined.loc[:, "psfMagDiffChi"] = psfMagDiffChiDf
dataJoined.loc[:, "raDiffMas"] = raDiffMasDf
dataJoined.loc[:, "decDiffMas"] = decDiffMasDf
dataJoined.loc[:, "raCosDecDiffMas"] = raCosDecDiffMasDf

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
print(matchedObjIdList)

### Create a pandas dataframe from all objects

In [ ]:
df_obj = pd.DataFrame()

In [ ]:
df_obj["objindex"] = matchedObjIdList
df_obj["ra"] = raSourcesCliAll
df_obj["dec"] = decSourcesCliAll
df_obj["nSources"] = nSourcesCliAll
df_obj["psfSn"] = psfSnAll
df_obj["psfMag"] = psfMagMeanMagAll
df_obj["psfMagErr"] = psfMagMeanMagErrAll
df_obj["psfMagStdMmag"]  =   psfMagStdMmagAll
df_obj["ap35InstMeanMag"] = ap35InstMagMeanMagAll
df_obj["ap35InstMeanMagErr"] = ap35InstMagMeanMagErrAll
df_obj["ap50InstMeanMag"] = ap50InstMagMeanMagAll
df_obj["ap50InstMeanMagErr"] = ap50InstMagMeanMagErrAll


In [ ]:
df_obj.sort_values('psfMag',ascending=True,inplace=True)

In [ ]:
df_obj

In [ ]:
df_obj.to_csv(output_objects_file)

### Example of source in object table

In [ ]:
dataJoined[dataJoined.index == 0]

In [ ]:
dataJoined.head()

In [ ]:
dataJoined.columns

In [ ]:
dataJoined["psfMagDiffMmag"]

## Plot the per object source 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,6))
ax.set_title("{}\n{} {} (max visit: {})".format(collection, tract, band, max(dataJoined["visit"])), fontsize=8)
nBins = 16 if len(dataJoined) < 4000 else 100
histData = psfMagStdMmagAll
xLabelStr = "std(psfFlux - psfFluxMean) ({} mmag)".format(band)
medianPsfFluxStd = np.median(histData)
n, bins, patches = ax.hist(histData, bins=nBins, density=True, histtype="step", lw=2, label="data (N = {})".format(len(histData)))
ax.axvline(x=medianPsfFluxStd, color="gray", linestyle='--', linewidth=1, label="median: {:.2f}".format(medianPsfFluxStd))

ax.tick_params(labelsize=7)
ax.set_xlabel(xLabelStr, fontsize=9)
ax.set_ylabel("Normalized (PDF)".format(band), fontsize=9)

ax.legend(fontsize=8)
if doSaveFigs:
    filename = "{}psfFluxStdHist_{}_{}_{}".format(saveDir, tract, band, collectionStr)
    print("Saving file in: {}".format(filename))
    fig.savefig(filename, dpi=150)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,6))
ax.set_title("{}\n{} {} (max visit: {})".format(collection, tract, band, max(dataJoined["visit"])), fontsize=8)

medianRaStd = np.median(raStdMasAll)
n, bins, patches = ax.hist(raStdMasAll, bins=nBins, density=True, histtype="step", color="tab:blue", lw=2, label="RA (N = {})".format(len(raStdMasAll)))

medianDecStd = np.median(decStdMasAll)
n, bins, patches = ax.hist(decStdMasAll, bins=nBins, density=True, histtype="step", color="tab:orange", lw=2, label="Dec (N = {})".format(len(decStdMasAll)))

medianRaCosDecStd = np.median(raCosDecStdMasAll)
n, bins, patches = ax.hist(raCosDecStdMasAll, bins=nBins, density=True, histtype="step", color="tab:green", lw=2, label="RA*cos(Dec) (N = {})".format(len(raCosDecStdMasAll)))

ax.axvline(x=medianRaStd, linestyle='--', color="tab:blue", linewidth=1, label="median: {:.2f} (mas)".format(medianRaStd))
ax.axvline(x=medianDecStd, linestyle='--', color="tab:orange", linewidth=1, label="median: {:.2f} (mas)".format(medianDecStd))
ax.axvline(x=medianRaCosDecStd, linestyle='--', color="tab:green", linewidth=1, label="median: {:.2f} (mas)".format(medianRaCosDecStd))


ax.tick_params(labelsize=7)
ax.set_xlabel("std(value - valueMean) ({} mas)".format(band), fontsize=9)
ax.set_ylabel("Normalized (PDF)".format(band), fontsize=9)

# ax.set_xlim(0, 400)
ax.legend(fontsize=8)
if doSaveFigs:
    filename = "{}RaDecStdHist_{}_{}_{}".format(saveDir, tract, band, collectionStr)
    print("Saving file in: {}".format(filename))
    fig.savefig(filename, dpi=150)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
# ax.set_title("{} {} {}".format(collection, tract, band), fontsize=8)
fig.suptitle("{}\n{} {} (max visit: {})".format(collection, tract, band, max(dataJoined["visit"])),
             fontsize=10, y=0.98)
ax1.axvline(x=0.0, color="gray", linestyle='--', linewidth=1)

n, bins, patches = ax1.hist(dataJoined["psfMagDiffMmag"], bins=2*nBins, density=True, histtype="step", lw=2, label="data")

# Gaussian fit of all data
(mu, sigma) = scipy.stats.norm.fit(dataJoined["psfMagDiffMmag"])
yFit = scipy.stats.norm.pdf(bins, mu, sigma)
ax1.plot(bins, yFit, "--", lw=1, fillstyle="full",
        label="Gaussian fit all\n(N={}) \n$\mu$={:.2f} (mmag)\n$\sigma$={:.2f}(mmag)".format(len(dataJoined), mu, sigma))
ax1.fill_between(bins, yFit, alpha=0.2)

# Gassian fit of 3-sigma clipped data
clippedDiffArray = scipy.stats.sigmaclip(dataJoined["psfMagDiffMmag"], low=3.0, high=3.0).clipped

(muClipped, sigmaClipped) = scipy.stats.norm.fit(clippedDiffArray)
yFitClipped = scipy.stats.norm.pdf(bins, muClipped, sigmaClipped)
ax1.plot(bins, yFitClipped, "--", lw=1, fillstyle="full",
        label="3-$\sigma$-clipped\n(N={}) \n$\mu$={:.2f} (mmag) \n$\sigma$={:.2f}(mmag)".format(len(clippedDiffArray), muClipped, sigmaClipped))
ax1.fill_between(bins, yFitClipped, alpha=0.2)

ax1.tick_params(labelsize=7)
ax1.set_xlabel("psfFlux - psfFluxMean ({} mmag)".format(band), fontsize=9)
ax1.set_ylabel("Normalized (PDF)".format(band), fontsize=9)

ax1.set_xlim(-200, 200)
ax1.legend(fontsize=12,loc="upper left")

# Plot chi histogram
ax2.axvline(x=0.0, color="gray", linestyle='--', linewidth=1)
n, bins, patches = ax2.hist(dataJoined["psfMagDiffChi"], bins=2*nBins, density=True, histtype="step", lw=2, label="data")

# Gaussian fit of all data
(mu, sigma) = scipy.stats.norm.fit(dataJoined["psfMagDiffChi"])
yFit = scipy.stats.norm.pdf(bins, mu, sigma)
ax2.plot(bins, yFit, "--", lw=1, fillstyle="full",
        label="Gaussian fit all\n(N={}) \n$\mu$={:.2f}\n$\sigma$={:.2f}".format(len(dataJoined), mu, sigma))
ax2.fill_between(bins, yFit, alpha=0.2)

# Gassian fit of 3-sigma clipped data
clippedDiffArray = scipy.stats.sigmaclip(dataJoined["psfMagDiffChi"], low=3.0, high=3.0).clipped
(muClipped, sigmaClipped) = scipy.stats.norm.fit(clippedDiffArray)
yFitClipped = scipy.stats.norm.pdf(bins, muClipped, sigmaClipped)
ax2.plot(bins, yFitClipped, "--", lw=1, fillstyle="full",
        label="3-$\sigma$-clipped\n(N={}) \n$\mu$={:.2f}\n$\sigma$={:.2f}".format(len(clippedDiffArray), muClipped, sigmaClipped))
ax2.fill_between(bins, yFitClipped, alpha=0.2)

ax2.tick_params(labelsize=7)
ax2.set_xlabel("$\chi$ = (psfMagDiff)/(psfMagErr$^2$ + psfMagMeanErr$^2$)$^{1/2}$", fontsize=9)
ax2.set_ylabel("Normalized (PDF)".format(band), fontsize=9)

ax2.set_xlim(-20, 20)
ax2.legend(fontsize=12,loc="upper left")

if doSaveFigs:
    filename = "{}psfFluxDiffHist_{}_{}_{}{}".format(saveDir, tract, band, collectionStr, maxVisitStr)
    print("Saving file in: {}".format(filename))
    fig.savefig(filename, dpi=150)

In [ ]:
band

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(6,5))
# ax.set_title("{} {} {}".format(collection, tract, band), fontsize=8)
fig.suptitle("{}\n{} {} (max visit: {})".format(collection, tract, band, max(dataJoined["visit"])),
             fontsize=10, y=0.98)
ax1.axvline(x=0.0, color="gray", linestyle='--', linewidth=1)

n, bins, patches = ax1.hist(dataJoined["psfMagDiffMmag"], bins=2*nBins, density=True, histtype="step", lw=2, label="data")

# Gaussian fit of all data
(mu, sigma) = scipy.stats.norm.fit(dataJoined["psfMagDiffMmag"])
yFit = scipy.stats.norm.pdf(bins, mu, sigma)
ax1.plot(bins, yFit, "--", lw=1, fillstyle="full",
        label="Gaussian fit all\n(N={}) \n$\mu$={:.2f} (mmag)\n$\sigma$={:.2f}(mmag)".format(len(dataJoined), mu, sigma))
ax1.fill_between(bins, yFit, alpha=0.2)

# Gassian fit of 3-sigma clipped data
clippedDiffArray = scipy.stats.sigmaclip(dataJoined["psfMagDiffMmag"], low=3.0, high=3.0).clipped

(muClipped, sigmaClipped) = scipy.stats.norm.fit(clippedDiffArray)
yFitClipped = scipy.stats.norm.pdf(bins, muClipped, sigmaClipped)
ax1.plot(bins, yFitClipped, "--", lw=1, fillstyle="full",
        label="3-$\sigma$-clipped\n(N={}) \n$\mu$={:.2f} (mmag) \n$\sigma$={:.2f}(mmag)".format(len(clippedDiffArray), muClipped, sigmaClipped))
ax1.fill_between(bins, yFitClipped, alpha=0.2)

ax1.tick_params(labelsize=7)
ax1.set_xlabel("psfFlux - psfFluxMean ({} mmag)".format(band), fontsize=9)
ax1.set_ylabel("Normalized (PDF)".format(band), fontsize=9)

ax1.set_xlim(-200, 200)
ax1.legend(fontsize=12,loc="upper right")
ax1.text(0.1, 0.95, "AUXTEL", transform=ax1.transAxes, fontsize=12,color="red",verticalalignment='top', bbox=props)
figname = f"photomrepeat_band_{band}_auxtel.png"
plt.savefig(figname)
plt.show()

In [ ]:
Band_To_Cmap_Dict = {"g":plt.cm.Greens,"r":plt.cm.Reds,"i":plt.cm.Oranges,"z":plt.cm.Purples,"y":plt.cm.Greys}

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
# ax.set_title("{} {} {}".format(collection, tract, band), fontsize=8)
fig.suptitle("{}\n{} {} (max visit: {})".format(collection, tract, band, max(dataJoined["visit"])),
             fontsize=10, y=0.98)
ax1.axvline(x=0.0, color="gray", linestyle='--', linewidth=1)

#n, bins, patches = ax1.hist(dataJoined["psfMagDiffMmag"], bins=2*nBins, density=True, histtype="step", lw=2, label="data")
ax1.hist2d(dataJoined["psfMag"],dataJoined["psfMagDiffMmag"], bins=max(2*nBins,100), cmap=Band_To_Cmap_Dict[band], label="data")
ax1.tick_params(labelsize=7)
ax1.set_ylabel("psfFlux - psfFluxMean ({} mmag)".format(band), fontsize=9)
ax1.set_xlabel("psfMag ({})mag ".format(band), fontsize=9)

ax1.set_ylim(-100, 100)
ax1.legend(fontsize=8)
ax1.grid()

# Plot chi histogram
ax2.axvline(x=0.0, color="gray", linestyle='--', linewidth=1)
ax2.hist2d(dataJoined["psfMag"],dataJoined["psfMagDiffChi"], bins=max(2*nBins,100),cmap=Band_To_Cmap_Dict[band], lw=2, label="data")


ax2.tick_params(labelsize=7)
ax2.set_ylabel("$\chi$ = (psfMagDiff)/(psfMagErr$^2$ + psfMagMeanErr$^2$)$^{1/2}$", fontsize=9)
ax2.set_xlabel("psfMag ({})mag ".format(band), fontsize=9)

ax2.set_ylim(-10, 10)
ax2.legend(fontsize=8)
ax2.grid()

if doSaveFigs:
    filename = "{}psfFluxDiffHist2D_{}_{}_{}{}".format(saveDir, tract, band, collectionStr, maxVisitStr)
    print("Saving file in: {}".format(filename))
    fig.savefig(filename, dpi=150)

## Classify the object into two categories according the value their psfMag dispersion (low or high)

In [ ]:
# Split out the smallest and largest PSF mag std objects (just to easily look at the extremes).
# Large dispersion sample
largeStdList = [index for index in psfMagStdMmagDict if psfMagStdMmagDict[index] >= 1.4*medianPsfFluxStd]
smallStdList= [index for index in psfMagStdMmagDict if psfMagStdMmagDict[index] < 0.4*medianPsfFluxStd]
for objectIndex in largeStdList:
    print("Large Std: objectIndex = {}  number of matches = {}".format(objectIndex, len(dataJoined[dataJoined["obj_index"] == objectIndex])))
# Low dispersion sample
for objectIndex in smallStdList:
    print("Small Std: objectIndex = {}  number of matches = {}".format(objectIndex, len(dataJoined[dataJoined["obj_index"] == objectIndex])))

## Fetch calexp to extract cutout

In [ ]:
# Plot cutouts centered on matched sources from different visits for a couple of objects.
# Selecting from the small/largeStdLists made above to look at the extremes.
imMin, imMax, Q = -0.001, 0.004, 8
expMin, expMax = -25, 100
numSub = 8
for maxBoxSize in [71]:
    plt.close("all")
    for objInd in smallStdList[0:3] + largeStdList[1:2]:
        joinedObjects = dataJoined[dataJoined["obj_index"] == objInd]
        numObj = len(joinedObjects)
        print("numObj = {} [with obj_index = {}]".format(numObj, objInd))
        nRow = 1 if numObj <= numSub else (numObj // numSub) + 1
        # print("nRow = {}".format(nRow))
        fig, axes = plt.subplots(nRow, numSub, figsize=(3*numSub, 3*nRow), constrained_layout=True)
        joinedObjects = dataJoined[dataJoined["obj_index"] == objInd]
        
        psfMag = joinedObjects["psfMag"]
        psfMagErr = joinedObjects["psfMagErr"]
        psfMagDiffMmag = joinedObjects["psfMagDiffMmag"].values
        psfMagStdMmag = psfMagStdMmagDict[objInd]

        clippedMeanRaArray = scipy.stats.sigmaclip(joinedObjects["ra"], low=3.0, high=3.0).clipped
        clippedMeanRa = np.mean(clippedMeanRaArray)
        clippedMeanDecArray = scipy.stats.sigmaclip(joinedObjects["dec"], low=3.0, high=3.0).clipped
        clippedMeanDec = np.mean(clippedMeanDecArray)

        iObj = -1
        for ax in axes.ravel():
            iObj += 1
            if iObj < numObj:
                row = joinedObjects[iObj:iObj + 1]
                detector = row["detector"].values[0]
                visit = row["visit"].values[0]
                psfMag = (row["psfFlux"].values[0]*u.nJy).to(u.ABmag)
                # build the data id to access to the calexp
                dataId = {"detector": detector, "visit": visit}
                exp = butler.get("calexp", dataId)
                wcs = exp.wcs
                pt = geom.SpherePoint(geom.Angle(row["ra"].values[0], geom.degrees),
                                      geom.Angle(row["dec"].values[0], geom.degrees))
                (xSrc, ySrc) = wcs.skyToPixel(pt)
                #define the bounding box
                boxSize = int(min(maxBoxSize, xSrc, exp.getWidth()-xSrc, ySrc, exp.getHeight()-ySrc) - 1)

                minBbox = geom.Point2I(int(xSrc) - boxSize ,int(ySrc) - boxSize)
                maxBbox = geom.Point2I(int(xSrc) + boxSize, int(ySrc) + boxSize)
                srcBbox = geom.Box2I(minBbox, maxBbox)

                # Make the cutout
                subimg = afwImage.ExposureF(exp, srcBbox, afwImage.PARENT, True)
                subimgRgb = afwRgb.makeRGB(subimg.image.array, None, None, expMin, expMax - expMin, Q)

                # Do imshow without lower ???
                im = ax.imshow(subimgRgb, interpolation="nearest", origin="lower")
                ax.text(0.5, 0.06, "psfSig: {:.2f} pix".format(row["psfSigma"].values[0]),
                        color="cyan", fontsize=14, ha="center", va="center", transform=ax.transAxes)
                ax.text(0.5, 0.15, "{:.2f}".format(psfMag),
                        color="violet", fontsize=14, ha="center", va="center", transform=ax.transAxes)

                diffColor = "red" if np.abs(psfMagDiffMmag[iObj]) > 100 else "lawngreen"
                ax.text(0.5, 0.92, "diff: {:.2f} mmag".format(psfMagDiffMmag[iObj]),
                        color=diffColor, fontsize=14, ha="center", va="center", transform=ax.transAxes)

                ax.set_title("{} {}sec".format(visit, row["expTime"].values[0]), fontsize=12)

        color = "green"
        plt.text(0.5, 0.84, "std: {:.2f} mmag".format(psfMagStdMmag), color=color, fontsize=14,
                 ha="center", va="center", transform=ax.transAxes)

        if doSaveFigs:
            filename = "{}{}_allOjbectsSmallStd_{}_{}_{}".format(saveDir, camera.getName(), objInd, maxBoxSize, collectionStr)
            print("Saving file in: {}".format(filename))
            plt.savefig(filename, dpi=250)

In [ ]:
# Plot all light curves (or as many as you include from stdList)
stdList = [index for index in psfMagStdMmagDict]
# Make light curves for both:
#     psfInstMag: the "instrument" magnitude
#     psfMag: the calibrated magnitude
for magStr in ["psfInstMag", "psfMag"]:
    fig, ax = plt.subplots(1, 1, figsize=(14, 10))
    for objInd in stdList[:25]:
        joinedObjects = dataJoined[dataJoined["obj_index"] == objInd]
        xData = joinedObjects["expMidptMjd"]
        yData = joinedObjects[magStr]
        yDataErr = joinedObjects["psfMagErr"]
        yDataSorted = [y for _, y in sorted(zip(xData, yData))]
        yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
        xDataSorted = sorted(xData)
        ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", alpha=0.6, label="{}".format(objInd))
        ax.scatter(xDataSorted, yDataSorted, s=9, alpha=1.0)
    ax.legend(fontsize=6, loc="upper right")
    ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
    ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
    title = f"Light Curves {magStr} for tract {tract} in band {band} (AUXTEL)"
    ax.set_title(title)
    if doSaveFigs:
        filename = "{}{}_lightCurves_{}".format(saveDir, camera.getName(),collectionStr)
        print("Saving file in: {}".format(filename))
        plt.savefig(filename, dpi=150)
    ax.grid()

In [ ]:
# Plot light curves divided by photo Std
magStr = "psfMag"
largeThresh = min(2.0*medianPsfFluxStd, 0.8*max(psfMagStdMmagDict.values()))
smallThresh = max(0.5*medianPsfFluxStd, 0.2*min(psfMagStdMmagDict.values()))
mediumThreshLower = medianPsfFluxStd - 0.04*medianPsfFluxStd
mediumThreshUpper = medianPsfFluxStd + 0.04*medianPsfFluxStd
largeStdList = [index for index in psfMagStdMmagDict if psfMagStdMmagDict[index] > largeThresh]
mediumUpperStdList = [index for index in psfMagStdMmagDict if (psfMagStdMmagDict[index] < mediumThreshUpper and psfMagStdMmagDict[index] > medianPsfFluxStd)]
mediumLowerStdList = [index for index in psfMagStdMmagDict if (psfMagStdMmagDict[index] < medianPsfFluxStd and psfMagStdMmagDict[index] > mediumThreshLower)]
smallStdList = [index for index in psfMagStdMmagDict if psfMagStdMmagDict[index] < smallThresh]
titleStrList = ["psf diff std > {:.2f} mmag".format(largeThresh),
                "psf diff {:.2f} < std < {:.2f} mmag".format(medianPsfFluxStd, mediumThreshUpper),
                "psf diff {:.2f} < std < {:.2f} mmag".format(mediumThreshLower, medianPsfFluxStd),
                "psf diff std < {:.2f} mmag".format(smallThresh)]

fig, axes = plt.subplots(4, 1, figsize=(14, 20))
for stdList, titleStr, ax in zip([largeStdList[:10], mediumUpperStdList[:10], mediumLowerStdList[:10], smallStdList[:10]], titleStrList, axes):
    for objInd in stdList:
        joinedObjects = dataJoined[dataJoined["obj_index"] == objInd]
        xData = joinedObjects["expMidptMjd"]
        yData = joinedObjects[magStr]
        yDataErr = joinedObjects["psfMagErr"]
        yDataSorted = [y for _, y in sorted(zip(xData, yData))]
        yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
        xDataSorted = sorted(xData)
        ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", alpha=0.6, label="{}".format(objInd))
        ax.scatter(xDataSorted, yDataSorted, s=9, alpha=1.0)
    ax.legend(fontsize=6)
    ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
    ax.set_ylabel("{}_psf (mag)".format(band), fontsize=10)
    ax.set_title(titleStr, fontsize=12, pad=4)
    ax.grid()
suptitle = f"Light Curves {magStr} for tract {tract} in band {band} (AUXTEL)"
plt.suptitle(suptitle)
plt.tight_layout()
if doSaveFigs:
    filename = "{}{}_lightCurves_{}".format(saveDir, camera.getName(),collectionStr)
    print("Saving file in: {}".format(filename))
    plt.savefig(filename, dpi=150)
plt.tight_layout()

# Select the objects

In [ ]:
#index_min = min(psfMagMeanMagDict, key=psfMagMeanMagDict.get)
index_min, magmin = min(psfMagMeanMagDict.items(), key=lambda x: x[1]) 

In [ ]:
index_min, magmin 

In [ ]:
psfMagStdMmagDict[index_min]

In [ ]:
magStr

In [ ]:
index_sel = index_min

In [ ]:
the_selected_object = objDataList[index_sel]
the_selected_object

In [ ]:
row_obj = df_obj[df_obj.objindex==index_sel]

In [ ]:
#output_objects_file = f"objectTable-t{tract}-b{band}-{collectionStr}.csv"
#output_objects_file_truncated = f"objectTable-t{tract}-b{band}"
this_object_output_file = 'sources_'+output_objects_file.replace(output_objects_file_truncated,output_objects_file_truncated+f"-o{index_sel}")   

In [ ]:
output_objects_file

In [ ]:
the_selected_object.to_csv(this_object_output_file)

In [ ]:
row_obj 

In [ ]:
for magStr in ["psfInstMag", "psfMag","apFlux_35_0_calMag","apFlux_50_0_calMag","localPhotoCalib"]:  
    magerrStr = magStr+"Err"
    fig, ax = plt.subplots(1, 1, figsize=(14, 2))   
    xData = the_selected_object["expMidptMjd"]
    yData = the_selected_object[magStr]
    yDataErr = the_selected_object[magerrStr]
    yDataSorted = [y for _, y in sorted(zip(xData, yData))]
    yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
    xDataSorted = sorted(xData)
    
    ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", color="b",alpha=0.6, label=f"{magStr}".format(objInd))
    ax.scatter(xDataSorted, yDataSorted, s=9, c='r',alpha=1.0)
    ax.grid()

    
    ax.legend(fontsize=6, loc="upper right")
    ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
    ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
    title = f"Light Curve for object {index_sel}  {magStr} for tract {tract} in band {band} (AUXTEL)"
    ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 2))   
xData = the_selected_object["expMidptMjd"]
yData = the_selected_object["apFlux_50_0_calMag"] - the_selected_object["apFlux_35_0_calMag"]
label = "apFlux_50_0_calMag - apFlux_35_0_calMag"
yDataErr = np.sqrt(the_selected_object["apFlux_35_0_calMagErr"]**2 + the_selected_object["apFlux_50_0_calMagErr"]**2)
yDataSorted = [y for _, y in sorted(zip(xData, yData))]
yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
xDataSorted = sorted(xData)
ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", color="b",alpha=0.6, label=label.format(objInd))
ax.scatter(xDataSorted, yDataSorted, s=9, c='r',alpha=1.0)
ax.grid()
ax.legend(fontsize=6, loc="upper right")
ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
title = f"Light Curve for object {index_sel} {label} for tract {tract} in band {band} (AUXTEL)"
ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 2))   
xData = the_selected_object["expMidptMjd"]
yData = the_selected_object["psfMag"] - the_selected_object["apFlux_35_0_calMag"]
label = "psfMag - apFlux_35_0_calMag"
yDataErr = np.sqrt(the_selected_object["apFlux_35_0_calMagErr"]**2 + the_selected_object["psfMagErr"]**2)
yDataSorted = [y for _, y in sorted(zip(xData, yData))]
yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
xDataSorted = sorted(xData)
ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", color="b",alpha=0.6, label=label.format(objInd))
ax.scatter(xDataSorted, yDataSorted, s=9, c='r',alpha=1.0)
ax.grid()
ax.legend(fontsize=6, loc="upper right")
ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
title = f"Light Curve for object {index_sel} {label} for tract {tract} in band {band} (AUXTEL)"
ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 2))   
xData = the_selected_object["expMidptMjd"]
yData = the_selected_object["psfMag"] - the_selected_object["apFlux_50_0_calMag"]
label = "psfMag - apFlux_50_0_calMag"
yDataErr = np.sqrt(the_selected_object["apFlux_50_0_calMagErr"]**2 + the_selected_object["psfMagErr"]**2)
yDataSorted = [y for _, y in sorted(zip(xData, yData))]
yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
xDataSorted = sorted(xData)
ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", color="b",alpha=0.6, label=label.format(objInd))
ax.scatter(xDataSorted, yDataSorted, s=9, c='r',alpha=1.0)
ax.grid()
ax.legend(fontsize=6, loc="upper right")
ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
title = f"Light Curve for object {index_sel} {label} for tract {tract} in band {band} (AUXTEL)"
ax.set_title(title)

In [ ]:
the_selected_object["visit"]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 3))   
the_selected_object.plot.scatter(x="visit",y="psfMag",s=5.,c="seeing",ax=ax,cmap="jet",rot=45)


In [ ]:
type(the_selected_object)

In [ ]:
index_sel = 912

In [ ]:
the_selected_object = objDataList[index_sel]
the_selected_object

In [ ]:
row_obj = df_obj[df_obj.objindex==index_sel]

In [ ]:
#output_objects_file = f"objectTable-t{tract}-b{band}-{collectionStr}.csv"
#output_objects_file_truncated = f"objectTable-t{tract}-b{band}"
#this_object_output_file = output_objects_file.replace(output_objects_file_truncated,output_objects_file_truncated+f"-o{index_sel}")   
this_object_output_file = 'sources_'+output_objects_file.replace(output_objects_file_truncated,output_objects_file_truncated+f"-o{index_sel}")   

In [ ]:
the_selected_object.to_csv(this_object_output_file)

In [ ]:
row_obj = df_obj[df_obj.objindex==index_sel]

In [ ]:
#for magStr in ["psfInstMag", "psfMag"]:   
for magStr in ["psfInstMag", "psfMag","apFlux_35_0_calMag","apFlux_50_0_calMag","localPhotoCalib"]:  
    magerrStr = magStr+"Err"
    fig, ax = plt.subplots(1, 1, figsize=(14, 2))   
    xData = the_selected_object["expMidptMjd"]
    yData = the_selected_object[magStr]
    yDataErr = the_selected_object[magerrStr]
    yDataSorted = [y for _, y in sorted(zip(xData, yData))]
    yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
    xDataSorted = sorted(xData)
    ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", color="b",alpha=0.6, label="{}".format(objInd))
    ax.scatter(xDataSorted, yDataSorted, s=9, c='r',alpha=1.0)
    ax.legend(fontsize=6, loc="upper right")
    ax.grid()
    ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
    ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
    #title = f"Light Curves {magStr} for tract {tract} in band {band} (AUXTEL)"
    title = f"Light Curve for object {index_sel}  {magStr} for tract {tract} in band {band} (AUXTEL)"
    ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 2))   
xData = the_selected_object["expMidptMjd"]
yData = the_selected_object["apFlux_50_0_calMag"] - the_selected_object["apFlux_35_0_calMag"]
label = "apFlux_50_0_calMag - apFlux_35_0_calMag"
yDataErr = np.sqrt(the_selected_object["apFlux_35_0_calMagErr"]**2 + the_selected_object["apFlux_50_0_calMagErr"]**2)
yDataSorted = [y for _, y in sorted(zip(xData, yData))]
yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
xDataSorted = sorted(xData)
ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", color="b",alpha=0.6, label=label.format(objInd))
ax.scatter(xDataSorted, yDataSorted, s=9, c='r',alpha=1.0)
ax.grid()
ax.legend(fontsize=6, loc="upper right")
ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
title = f"Light Curve for object {index_sel} {label} for tract {tract} in band {band} (AUXTEL)"
ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 2))   
xData = the_selected_object["expMidptMjd"]
yData = the_selected_object["psfMag"] - the_selected_object["apFlux_35_0_calMag"]
label = "psfMag - apFlux_35_0_calMag"
yDataErr = np.sqrt(the_selected_object["apFlux_35_0_calMagErr"]**2 + the_selected_object["psfMagErr"]**2)
yDataSorted = [y for _, y in sorted(zip(xData, yData))]
yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
xDataSorted = sorted(xData)
ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", color="b",alpha=0.6, label=label.format(objInd))
ax.scatter(xDataSorted, yDataSorted, s=9, c='r',alpha=1.0)
ax.grid()
ax.legend(fontsize=6, loc="upper right")
ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
title = f"Light Curve for object {index_sel} {label} for tract {tract} in band {band} (AUXTEL)"
ax.set_title(title)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 2))   
xData = the_selected_object["expMidptMjd"]
yData = the_selected_object["psfMag"] - the_selected_object["apFlux_50_0_calMag"]
label = "psfMag - apFlux_50_0_calMag"
yDataErr = np.sqrt(the_selected_object["apFlux_50_0_calMagErr"]**2 + the_selected_object["psfMagErr"]**2)
yDataSorted = [y for _, y in sorted(zip(xData, yData))]
yDataErrSorted = [y for _, y in sorted(zip(xData, yDataErr))]
xDataSorted = sorted(xData)
ax.errorbar(xDataSorted, yDataSorted, yerr=yDataErrSorted, linestyle="-", color="b",alpha=0.6, label=label.format(objInd))
ax.scatter(xDataSorted, yDataSorted, s=9, c='r',alpha=1.0)
ax.grid()
ax.legend(fontsize=6, loc="upper right")
ax.set_xlabel("{}{}".format("expMidpdMjd", " (MJD)"), fontsize=10)
ax.set_ylabel("{}_{}".format(band, magStr), fontsize=10)
title = f"Light Curve for object {index_sel} {label} for tract {tract} in band {band} (AUXTEL)"
ax.set_title(title)